<a href="https://colab.research.google.com/github/MrsIgnis/MMO_tasks/blob/main/MOCI_task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IV. Решение задач. Классификация. Кластеризация. Генерация

Задание: используя полученные знания, пройти этапы с *collect data* до *split data*. Т. е. подобрать датасет (в котором будут текста и их категории), загрузить, провести предоработку, проанализировать, векторизовать, провести кластеризацию, сравнить результаты с реальной разметкой и в результате разбить на train, test и val выборки.




---





> Для кластеризации я выбрала BERTopic, поэтому этап предоработки данных более мягкий в сравнении с предыдущими л/р, а отдельный этап векторизации не нужен (BERTopic сам может это сделать).



**0. Загрузка библиотек**

In [ ]:
!pip install nltk pymorphy3
!pip install bertopic sentence-transformers hdbscan umap-learn

In [ ]:
import pandas as pd
import math
import re
import nltk
import pymorphy3
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
stop_words_en = set(stopwords.words("english"))

In [ ]:
lemma_en = WordNetLemmatizer()

In [ ]:
df = pd.read_csv('/content/pokemon-cards.csv')
df

In [ ]:
df = df.rename(columns={'caption': 'text', 'set_name': 'category'})
df

**I. Предоработка датасета**

In [ ]:
def is_english(word: str) -> bool:
    return bool(re.search('[a-z]', word, re.IGNORECASE))

In [ ]:
def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^а-яёa-z\s]', '', text, flags=re.IGNORECASE)
    words = word_tokenize(text)
    lemmas = []
    for word in words:
        if word not in stop_words_en:
            if is_english(word):
                lemma = lemma_en.lemmatize(word)
            else:
                lemma = word
            lemmas.append(lemma)
    return ' '.join(lemmas)

In [ ]:
df['processed_text'] = df['text'].apply(preprocess_text)

In [ ]:
print("--- Данные после предобработки: ---")
df[['text', 'processed_text', 'category']]

**II. Анализ датасета**

In [ ]:
category_counts = df['category'].value_counts()
print("Распределение по категориям:\n")
print(category_counts)

In [ ]:
plt.figure(figsize=(50, 10))
sns.barplot(x=category_counts.index, y=category_counts.values)
plt.title("Распределение текстов по категориям")
plt.xlabel("Категория")
plt.ylabel("Количество текстов")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df['text_len_raw'] = df['text'].apply(len)
df['text_len_processed'] = df['processed_text'].apply(len)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(df['text_len_raw'], kde=True)
plt.title("Распределение длины исходных текстов")
plt.xlabel("Длина текста")
plt.ylabel("Частота")

In [ ]:
plt.subplot(1, 2, 2)
sns.histplot(df['text_len_processed'], kde=True)
plt.title("Распределение длины обработанных текстов")
plt.xlabel("Длина текста")
plt.ylabel("Частота")
plt.tight_layout()
plt.show()

In [ ]:
all_words = " ".join(df['processed_text']).split()
word_counts = Counter(all_words)
most_common_words = word_counts.most_common(15)
print("Топ-15 наиболее частых слов (после предобработки):")
print(most_common_words)

In [ ]:
if most_common_words:
    common_words_df = pd.DataFrame(most_common_words, columns=['word', 'count'])
    plt.figure(figsize=(10, 6))
    sns.barplot(x='count', y='word', data=common_words_df, palette='viridis')
    plt.title("Топ-15 наиболее частых слов")
    plt.xlabel("Частота")
    plt.ylabel("Слово")
    plt.tight_layout()
    plt.show()
else:
    print("Недостаточно слов для отображения частотности после предобработки (возможно, датасет слишком мал или все слова - стоп-слова).")

**III. Кластеризация**

In [ ]:
documents = df['processed_text'].tolist()
true_categories = df['category'].tolist()

In [ ]:
embedding_model_name = 'all-MiniLM-L6-v2'

In [ ]:
sentence_model = SentenceTransformer(embedding_model_name)

In [ ]:
hdbscan_model = HDBSCAN(
    min_cluster_size=3,
    min_samples=1,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)
umap_model = UMAP(
    n_neighbors=10,
    n_components=3,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

In [ ]:
topic_model = BERTopic(
        embedding_model=sentence_model,
        language="english",
        min_topic_size=3,
        nr_topics=150,
        hdbscan_model=hdbscan_model,
        umap_model=umap_model,
        calculate_probabilities=True,
        verbose=True
)
topics, probabilities = topic_model.fit_transform(documents)
df['bertopic_cluster_label'] = topics
print("Информация о найденных темах (BERTopic):")
topic_info_df = topic_model.get_topic_info()
if not topic_info_df.empty:
        print(topic_info_df)
else:
        print("BERTopic не нашел тем (кроме, возможно, выбросов).")


if -1 in set(topics) and len(set(topics)) == 1:
        print("\nBERTopic отнес все документы к выбросам (тема -1). Попробуйте изменить параметры, например, min_topic_size или настройки HDBSCAN/UMAP.")
else:
        for topic_id in sorted(list(set(topics) - {-1})):
            print(f"\nТема {topic_id}:")
            words = topic_model.get_topic(topic_id)
            if words:
                print([word for word, score in words[:15]])
            else:
                print("Нет репрезентативных слов для этой темы.")

**IV. Сравнение результатов кластеризации с реальной разметкой**

In [ ]:
if 'bertopic_cluster_label' in df.columns and df['bertopic_cluster_label'].nunique() > 0 and df['bertopic_cluster_label'].max() > -2:
    clustered_mask = df['bertopic_cluster_label'] != -1
    if clustered_mask.sum() > 0 and df.loc[clustered_mask, 'bertopic_cluster_label'].nunique() > 1 :
        true_labels_filtered = df.loc[clustered_mask, 'category']
        cluster_labels_filtered = df.loc[clustered_mask, 'bertopic_cluster_label']

        ari_bertopic = adjusted_rand_score(true_labels_filtered, cluster_labels_filtered)
        nmi_bertopic = normalized_mutual_info_score(true_labels_filtered, cluster_labels_filtered)
        homogeneity_bt = homogeneity_score(true_labels_filtered, cluster_labels_filtered)
        completeness_bt = completeness_score(true_labels_filtered, cluster_labels_filtered)
        v_measure_bt = v_measure_score(true_labels_filtered, cluster_labels_filtered)

        print(f"Adjusted Rand Index (ARI) for BERTopic: {ari_bertopic:.3f}")
        print(f"Normalized Mutual Information (NMI) for BERTopic: {nmi_bertopic:.3f}")
        print(f"Homogeneity (BERTopic): {homogeneity_bt:.3f}")
        print(f"Completeness (BERTopic): {completeness_bt:.3f}")
        print(f"V-measure (BERTopic): {v_measure_bt:.3f}")
        print(f"Количество документов, отнесенных к выбросам (тема -1): {(df['bertopic_cluster_label'] == -1).sum()}")

        contingency_table_bertopic = pd.crosstab(df['category'], df['bertopic_cluster_label'])
        print("\nТаблица сопряженности (реальные категории vs. BERTopic кластеры):")
        print(contingency_table_bertopic)
    else:
        print("Недостаточно кластеризованных данных (не считая выбросов) или кластеров для расчета метрик.")
        if 'bertopic_cluster_label' in df.columns:
             print(f"Уникальные метки кластеров BERTopic: {df['bertopic_cluster_label'].unique()}")
             print(f"Количество документов с меткой -1 (outliers): {(df['bertopic_cluster_label'] == -1).sum()}")
else:
    print("Кластеризация BERTopic не была выполнена или не дала результатов для сравнения.")

In [ ]:
if 'topic_model' in locals() and hasattr(topic_model, 'topics_') and -1 not in topic_model.get_topic_info()['Topic'].tolist() or len(topic_model.get_topic_info()) > 1:
    print("\nПопытка визуализации тем...")
    try:
        fig_hierarchy = topic_model.visualize_hierarchy(top_n_topics=50)
        if fig_hierarchy: fig_hierarchy.show()

        fig_topics = topic_model.visualize_topics()
        if fig_topics: fig_topics.show()

        fig_barchart = topic_model.visualize_barchart(top_n_topics=min(10, len(topic_model.get_topic_info())-1 if -1 in topic_model.get_topic_info()['Topic'].tolist() else len(topic_model.get_topic_info())))
        if fig_barchart: fig_barchart.show()

        fig_heatmap = topic_model.visualize_heatmap(top_n_topics=20)
        if fig_heatmap: fig_heatmap.show()

    except Exception as e:
        print(f"Ошибка при визуализации тем BERTopic: {e}")
else:
    print("\nМодель BERTopic не была успешно обучена или не нашла достаточно тем для основной визуализации.")


**V. Разбиение на train, test и val выборки**

In [ ]:
X = df['text']
y = df['category']

In [ ]:
min_samples_per_class = y.value_counts().min() if not y.empty else 0
stratify_option_y = None
stratify_option_y_train_val = None

In [ ]:
if not y.empty and min_samples_per_class >= 2 :
    stratify_option_y = y
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=stratify_option_y
    )
    if not y_train_val.empty and (y_train_val.value_counts().min() >= 2 or len(y_train_val.unique()) == 1):
        stratify_option_y_train_val = y_train_val
    else:
        print(f"\nПредупреждение для второго сплита: В train_val классах менее 2 образцов или выборка пуста. Стратификация для val не будет применяться.")
        stratify_option_y_train_val = None

    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=stratify_option_y_train_val
    )
elif not y.empty:
    print(f"\nПредупреждение: В некоторых классах менее 2 образцов (минимально: {min_samples_per_class}). Стратификация не применяется.")
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)
else:
    print("Целевая переменная y пуста. Разбиение на выборки не может быть выполнено.")
    X_train, X_val, X_test, y_train, y_val, y_test = (pd.Series(dtype='object'), pd.Series(dtype='object'), pd.Series(dtype='object'),
                                                      pd.Series(dtype='object'), pd.Series(dtype='object'), pd.Series(dtype='object'))


In [ ]:
print("--- Разбиение на выборки: ---")
print(f"Размер исходного X: {X.shape}, y: {y.shape}")
print(f"Размер X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размер X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Размер X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
if not y_train.empty: print("Распределение категорий в y_train:\n", y_train.value_counts(normalize=True))
if not y_val.empty: print("\nРаспределение категорий в y_val:\n", y_val.value_counts(normalize=True))
if not y_test.empty: print("\nРаспределение категорий в y_test:\n", y_test.value_counts(normalize=True))